In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install h2o4gpu

     |████████████████████████████████| 346.3MB 48kB/s 
     |████████████████████████████████| 460kB 49.1MB/s 
     |████████████████████████████████| 6.7MB 43.4MB/s 
  Created wheel for psutil: filename=psutil-5.7.2-cp36-cp36m-linux_x86_64.whl size=279868 sha256=6b952b7eb270d010d385e52424277851adb6a3f3aa43f9d3e4b4e01b5a36a81d
  Stored in directory: /root/.cache/pip/wheels/39/a0/f5/c4fa280463e29aea07797acb5312358fefb067c1f4f98e11b1
Successfully built psutil
  Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
from osgeo import gdal
import os
import numpy as np

#!pip install tf-nightly-gpu-2.0-preview

from time import time

import h5py
import random
from scipy import ndarray
from skimage import exposure
#import skimage as sk
#from skimage import transform
#from skimage import util
#from skimage.exposure import adjust_sigmoid, is_low_contrast
from random import random as rd
import cv2 as cv
#from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import re
import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator #, load_img, img_to_array
#from keras.applications.vgg16 import preprocess_input #, decode_predictions, VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras.losses import binary_crossentropy as bce
from keras.losses import categorical_crossentropy as cce
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping #,TensorBoard#, ReduceLROnPlateau, LearningRateScheduler
from keras.utils.np_utils import to_categorical   
from h2o4gpu.solvers.kmeans import KMeans as kmeans
##categorical_labels = to_categorical(int_labels, num_classes=3)


In [ ]:
'''
import datetime, os

logs_base_dir = "./logs"
os.makedirs(logs_base_dir, exist_ok=True)
'''
K.clear_session()



def shuffle_list(*ls, seeds =78):
  random.seed(seeds)
  l =list(zip(*ls))

  random.shuffle(l)
  return zip(*l)





def raster2array(raster, bnd,x,y,wndo):
    rr = np.zeros((wndo, wndo, bnd))
    newValue = 0
    for b in range(bnd):
        band = raster.GetRasterBand(b+1)
        rr[:,:,b] = band.ReadAsArray(x, y, wndo, wndo)
        #noDataValue = band.GetNoDataValue()
    #rr[rr == noDataValue] = newValue
    return rr.astype(np.uint8)
ipath = []
cpath = []

def sorted_aphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)
  
  
def get_all_images(folder, ext):

    all_files = []
    #Iterate through all files in folder
    for file in sorted_aphanumeric(os.listdir(folder)):
        #Get the file extension
        _,  file_ext = os.path.splitext(file)

        #If file is of given extension, get it's full path and append to list
        if ext in file_ext:
            full_file_path = os.path.join(folder, file)
            all_files.append(full_file_path)

    #Get list of all files
    return all_files


def coord(ipath, nm, hmargin, wmargin, steps):
    raster = gdal.Open(ipath)


    w,h = round(raster.RasterXSize), round(raster.RasterYSize)
    print(h,w)
    raster = None
    #margin = 0
    size = 256
    ar1= (np.arange(hmargin,h+1,size))
    ar1 = ar1[:-1]
    #print(ar1)
    ar2= np.arange(wmargin,w+1,size)
    ar2 = ar2[:-1]
    #print(ar2)

    i= (ar1.shape[0]) 
    j = (ar2.shape[0])
    l = i*j
    print(i,j,l)
    print(ar1.shape)
    print(ar2.shape)
    ar2 = ar2*np.ones((i,j))
    ar1 = ar1.reshape(i,1)*(np.ones((i,j)))
    ind = np.arange(0,l,1).reshape(i,j)
    idj = np.zeros_like(ind)
    idj = idj + nm
    ar1 = np.around(ar1)
    ar2 = np.around(ar2)


    step = steps
    ind = list(ind.ravel()[::step])
    ar1 = list(ar1.ravel()[::step])
    ar2 = list(ar2.ravel()[::step])
    idj = list(idj.ravel()[::step])

    return (idj,ind,ar1,ar2)


def rdata(start_w, start_h, w,h, wmargin, hmargin, number, i):
    
    np.random.seed(440+i)
    a1 = list(np.squeeze(np.random.randint(start_h,(h-hmargin),size=(number,1)).astype(np.float64)))
    np.random.seed(494+i)
    a2 = list(np.squeeze(np.random.randint(start_w,(w-wmargin),size=(number,1)).astype(np.float64)))
    in1 = list(np.squeeze(np.arange(0,number,1).reshape(number,1)))
    id1 = list(np.squeeze(np.zeros((number,1),dtype=np.uint8)+i))
    ##id1 = list(np.squeeze(np.zeros((number,1),dtype=np.uint8)+i))

    return (id1, in1, a1, a2)

def apply_rdata(filepath,idj1,ind1,ar11,ar21,number, unique_id):
    raster = gdal.Open(filepath)
    w,h = round(raster.RasterXSize), round(raster.RasterYSize)
    #print(h,w)
    raster = None

    idj2,ind2,ar12,ar22 = rdata(0,0, w, h,size,size,number,unique_id)   

    print('idj: ', len(idj2))
    print('ar1: ', len(ar12))
    print('ar2: ', len(ar22))
    
    idj1 = idj1 + idj2
    ind1 = ind1 + ind2
    ar11 = ar11 + ar12
    ar21 = ar21 + ar22

    return idj1,ind1, ar11, ar21


def apply_coord(filepath,idj1,ind1,ar11,ar21,steps, unique_id):
    raster = gdal.Open(filepath)
    w,h = round(raster.RasterXSize), round(raster.RasterYSize)
    #print(h,w)
    raster = None

    idj2,ind2,ar12,ar22 = coord(filepath,unique_id,0,0,steps)   
    
    print('idj: ', len(idj2))
    print('ar1: ', len(ar12))
    print('ar2: ', len(ar22))
    
    
    idj1 = idj1 + idj2
    ind1 = ind1 + ind2
    ar11 = ar11 + ar12
    ar21 = ar21 + ar22

    return idj1,ind1, ar11, ar21


In [ ]:
# Convert Raster to array

def img_generator(ipathi,cpathi,idjx,indx,ar1x,ar2x, batch):
    
    def generate_labels(image,ndvi,ndwi):
        img1 = process_kmeans2(image)
        img2 = np.zeros_like(img1)
        clt = norm_classes(img1,img2,ndvi,ndwi)
        return clt


    def process_kmeans2(imgn, K=4):
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 0.05)
        # Set flags (Just to avoid line break in the code)
        flags = cv.KMEANS_RANDOM_CENTERS  
        img1 = imgn.reshape((-1,imgn.shape[-1]))
        img1 = np.float32(img1)
        # compactness,label,center = cv.kmeans(img1, K,None,criteria,30,flags)
        kmn = kmeans(n_clusters=4, n_init=30, max_iter=10, tol=0.01, precompute_distances='auto', verbose=0, random_state=36, copy_x=True, n_jobs=None, algorithm='auto').fit(img1)
        label = kmn.labels_
        return label.reshape((imgn.shape[0],imgn.shape[1]))

    def norm_classes(img1,img2,ndvi,ndwi):
        for c in list(set(img1.flatten())):    
            mask = np.zeros_like(img1)
            mask[img1==c] =1
            mean_ndvi = np.sum(ndvi[img1==c])/np.sum(mask)
            mean_ndwi = np.sum(ndwi[img1==c])/np.sum(mask)
            if (mean_ndvi<0.2 and mean_ndvi>0 and mean_ndwi<0):
                img2[img1==c] = 7
            elif (mean_ndvi<0.4 and mean_ndvi>=0.2 and mean_ndwi<0):
                img2[img1==c] = 8
            elif (mean_ndvi>=0.4 and mean_ndwi<0):
                img2[img1==c] = 9
            elif (mean_ndvi<0 and mean_ndwi>=0.3):
                img2[img1==c] = 1 
            elif (mean_ndvi<0 and mean_ndwi<0.3 and mean_ndwi>0):
                img2[img1==c] = 2 
            elif (mean_ndvi<0 and mean_ndwi<0):
                img2[img1==c] = 3
            elif (mean_ndvi<=0.2 and mean_ndvi>0 and mean_ndwi<=0.3 and mean_ndwi>0):
                img2[img1==c] = 4 
            elif (mean_ndvi<=0.4 and mean_ndvi>0.2 and mean_ndwi<=0.3 and mean_ndwi>0):
                img2[img1==c] = 5 
            elif (mean_ndvi==0 and mean_ndwi==0):
                img2[img1==c] = 0
            else:
                img2[img1==c] = 6
        return img2

    def add_veg_water_index(image):
        h,w = image.shape[:2]
        image2= np.zeros((h,w,6))
        image2[:,:,:4] = image
        del image
        r = image2[:,:,2]
        g = image2[:,:,1]
        nir = image2[:,:,3]
        
        num1 = nir-r
        dnm1 = nir+r
        num2= g-nir
        dnm2 = g+nir
        
        del nir,r,g
        ndvi = np.divide(num1,dnm1, where = (dnm1!=0), dtype = np.float64)
        ndwi = np.divide(num2,dnm2, where = (dnm2!=0), dtype = np.float64)
        
        del num1,num2,dnm1,dnm2
        ndvi[ndvi>1] = 0
        ndvi[ndvi<-1] = 0
        ndwi[ndwi>1] = 0
        ndwi[ndwi<-1] = 0
        
        image2[:,:,4] = 127.5 + ndvi*127.5
        image2[:,:,5] = 127.5 + ndwi*127.5
        
        return image2,ndvi,ndwi

    def raster2arr(raster, bnd,x,y,wndo):
        rr = np.zeros((wndo, wndo, bnd))
        newValue = 0
        for b in range(bnd):
            band = raster.GetRasterBand(b+1)
            img = band.ReadAsArray(x, y, wndo, wndo)
            noDataValue = band.GetNoDataValue()
            img[img == noDataValue] = newValue
            rr[:,:,b] = img
        return rr.astype(np.uint8)
    
    def contrast(image_array: ndarray):
            v_min, v_max = np.percentile(image_array, (0.2, 99.8))
            return exposure.rescale_intensity(image_array, in_range=(v_min, v_max))

    def com_gen(image,label):
        K.set_image_data_format('channels_last')
        def combine_generator(gen1, gen2):
            while True:
                yield(gen1.next(), gen2.next())
        
        data_gen_args = dict(horizontal_flip=True,
                         vertical_flip=True,
                         rotation_range=45,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         #brightness_range=[0.8,1.2],
                         #shear_range=0.2,
                         #channel_shift_range = 20,
                         zoom_range=0.2,
                         fill_mode='nearest',
                         data_format = "channels_last",
                         rescale = 1./255)
        '''
        data_gen_args = dict(data_format = "channels_last",
                             rescale = 1./255)
        '''
        image_datagen = ImageDataGenerator(**data_gen_args)
        mask_datagen = ImageDataGenerator(**data_gen_args)
        #image_datagen.fit(image, augment=True, rounds=1, seed=42)
        #mask_datagen.fit(label, augment=True, rounds=1, seed=42)
        image_generator = image_datagen.flow(image, batch_size=1, seed=42)
        mask_generator = mask_datagen.flow(label, batch_size=1, seed=42)
        train_generator = combine_generator(image_generator,mask_generator)
        return train_generator

    
    ntr = len(ar1x)
    q =list(range(ntr))
    random.seed(948)
    random.shuffle(q)
    
    num = 0
    while (num<ntr):
        
        random.shuffle(q)
        
        batch_input = np.zeros((batch,size,size,6))
        batch_output = np.zeros((batch,size,size,10))
        #ind1,ar11,ar21 = shuffle_lst(ind1,ar11,ar21)
       
        for m in range(batch):
            ip = q[num]
            x = ar2x[ip]
            y = ar1x[ip]
            ijx = idjx[ip]
            
        
            raster = gdal.Open(ipathi[ijx])        
            img = raster2arr(raster,4,x,y,size)
            raster = None
            batch_input[m,:,:,:],ndvi,ndwi = add_veg_water_index(img)
            clt = generate_labels(batch_input[m,:,:,:],ndvi,ndwi)
            
            # raster = gdal.Open(cpathi[ijx])
             
            # clt = raster2arr(raster,1,x,y,size)
            # raster = None
            batch_output[m,:,:,:] = (to_categorical(clt,num_classes=10))*255.0
            del img,clt
        
        num += 1
        return (com_gen(batch_input, batch_output))
        #return (com_gen(contrast(batch_input), batch_output))

def msk_generator(ipathm,cpathm,idjy,indy,ar1y,ar2y, batch):    
    
    def generate_labels(image,ndvi,ndwi):
        img1 = process_kmeans2(image)
        img2 = np.zeros_like(img1)
        clt = norm_classes(img1,img2,ndvi,ndwi)
        return clt

    def process_kmeans2(imgn, K=4):
        criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 0.05)
        # Set flags (Just to avoid line break in the code)
        flags = cv.KMEANS_RANDOM_CENTERS  
        img1 = imgn.reshape((-1,imgn.shape[-1]))
        img1 = np.float32(img1)
        # compactness,label,center = cv.kmeans(img1, K,None,criteria,30,flags)
        kmn = kmeans(n_clusters=4, n_init=30, max_iter=10, tol=0.01, precompute_distances='auto', verbose=0, random_state=36, copy_x=True, n_jobs=None, algorithm='auto').fit(img1)
        label = kmn.labels_
        return label.reshape((imgn.shape[0],imgn.shape[1]))

    def norm_classes(img1,img2,ndvi,ndwi):
        for c in list(set(img1.flatten())):    
            mask = np.zeros_like(img1)
            mask[img1==c] =1
            mean_ndvi = np.sum(ndvi[img1==c])/np.sum(mask)
            mean_ndwi = np.sum(ndwi[img1==c])/np.sum(mask)
            if (mean_ndvi<0.2 and mean_ndvi>0 and mean_ndwi<0):
                img2[img1==c] = 7
            elif (mean_ndvi<0.4 and mean_ndvi>=0.2 and mean_ndwi<0):
                img2[img1==c] = 8
            elif (mean_ndvi>=0.4 and mean_ndwi<0):
                img2[img1==c] = 9
            elif (mean_ndvi<0 and mean_ndwi>=0.3):
                img2[img1==c] = 1 
            elif (mean_ndvi<0 and mean_ndwi<0.3 and mean_ndwi>0):
                img2[img1==c] = 2 
            elif (mean_ndvi<0 and mean_ndwi<0):
                img2[img1==c] = 3
            elif (mean_ndvi<=0.2 and mean_ndvi>0 and mean_ndwi<=0.3 and mean_ndwi>0):
                img2[img1==c] = 4 
            elif (mean_ndvi<=0.4 and mean_ndvi>0.2 and mean_ndwi<=0.3 and mean_ndwi>0):
                img2[img1==c] = 5 
            elif (mean_ndvi==0 and mean_ndwi==0):
                img2[img1==c] = 0
            else:
                img2[img1==c] = 6
        return img2

    def add_veg_water_index(image):
        h,w = image.shape[:2]
        image2= np.zeros((h,w,6))
        image2[:,:,:4] = image
        del image
        r = image2[:,:,2]
        g = image2[:,:,1]
        nir = image2[:,:,3]
        
        num1 = nir-r
        dnm1 = nir+r
        num2= g-nir
        dnm2 = g+nir
        
        del nir,r,g
        ndvi = np.divide(num1,dnm1, where = (dnm1!=0), dtype = np.float64)
        ndwi = np.divide(num2,dnm2, where = (dnm2!=0), dtype = np.float64)
        
        del num1,num2,dnm1,dnm2
        ndvi[ndvi>1] = 0
        ndvi[ndvi<-1] = 0
        ndwi[ndwi>1] = 0
        ndwi[ndwi<-1] = 0
        
        image2[:,:,4] = 127.5 + ndvi*127.5
        image2[:,:,5] = 127.5 + ndwi*127.5
        
        return image2,ndvi,ndwi


    def raster2arr(raster, bnd,x,y,wndo):
        rr = np.zeros((wndo, wndo, bnd))
        newValue = 0
        for b in range(bnd):
            band = raster.GetRasterBand(b+1)
            img = band.ReadAsArray(x, y, wndo, wndo)
            noDataValue = band.GetNoDataValue()
            img[img == noDataValue] = newValue
            rr[:,:,b] = img
        return rr.astype(np.uint8)

    def com_gen(image2,label2):
        K.set_image_data_format('channels_last')
        def combine_generator(gen1, gen2):
            while True:
                yield(gen1.next(), gen2.next())                                   
  
        data_gen_args2 = dict(data_format = "channels_last",
                             rescale = 1./255)
        image_datagen2 = ImageDataGenerator(**data_gen_args2)
        mask_datagen2 = ImageDataGenerator(**data_gen_args2)
        #image_datagen.fit(image, augment=True, rounds=5, seed=42)
        #mask_datagen.fit(label, augment=True, rounds=5, seed=42)
        image_generator2 = image_datagen2.flow(image2, batch_size=1, seed=56)
        mask_generator2 = mask_datagen2.flow(label2, batch_size=1, seed=56)
        val_generator = combine_generator(image_generator2,mask_generator2)
        return val_generator
    
    ntr = len(ar1y)
    q = list(range(ntr))
    random.seed(493)
    random.shuffle(q)
        
    num = 0
    while (num<ntr):
        
        #random.shuffle(q)
        
        batch_inputm = np.zeros((batch,size,size,6))
        batch_outputm = np.zeros((batch,size,size,10))
        #ind1,ar11,ar21 = shuffle_lst(ind1,ar11,ar21)
       
        for m in range(batch):
            ip = q[num]
            x = ar2y[ip]
            y = ar1y[ip]
            ijy = idjy[ip]
            
            raster = gdal.Open(ipathm[ijy])     
            img = raster2arr(raster,4,x,y,size)
            raster = None           
            batch_inputm[m,:,:,:],ndvi,ndwi = add_veg_water_index(img)
            clt = generate_labels(batch_inputm[m,:,:,:],ndvi,ndwi)
            # raster = gdal.Open(cpathm[ijy])
             
            # clt = raster2arr(raster,1,x,y,size)
            # raster = None  
            batch_outputm[m,:,:,:] = (to_categorical(clt,num_classes=10))*255.0  
            del img,clt
    
        num += 1
        return (com_gen(batch_inputm, batch_outputm))
        #yield (batch_inputm, batch_outputm)


In [ ]:
def iou(y_true, y_pred):
    smooth = 1e-12
    '''
    size =int(256)
    y_true = K.cast(K.reshape(y_true, (-1,size,size)),np.uint8)
    y_true = K.one_hot(y_true,3)
    '''
    intersection = K.sum((y_true * y_pred), axis = (0,1,2))
    #intrsctn = K.sum(intersection[1:])
    #intrsctn = K.sum(intersection[1]+intersection[2]+intersection[4])
    union = K.sum((y_true + y_pred), axis = (0,1,2)) - K.sum((y_true * y_pred), axis = (0,1,2))
    #unin = K.sum(union[1:])
    #unin = K.sum(union[1]+union[2]+union[4])
    
    iu = ((intersection + smooth)/ (union + smooth))
    #iu = K.sum((intrsctn+smooth)/(unin+smooth))
    return K.sum(iu[1:])#+iu[2]+iu[4]+iu[8]+iu[9])
    

def iou_loss(y_true, y_pred):
    ncl = 9.0
    return (ncl-iou(y_true, y_pred))


 
def dice(y_true, y_pred):
    smooth1 = 1e-12
    '''
    size = int(256)
    y_true = K.cast(K.reshape(y_true, (-1,size,size)),np.uint8)
    y_true = K.one_hot(y_true,3)
    #class_weights = K.sum(y_true, axis = (0,1,2,3))/(K.sum(y_true, axis = (0,1,2,3)) + K.sum((1 -y_true), axis = (0,1,2,3)))
    #y_pred = weighted_ypred(y_true,y_pred)
    '''
    #w1 = K.sum(y_true)
    #w2 = K.sum(1-y_true) 
    num1 = K.sum((y_true *  y_pred), axis = (0,1,2))
    dnm1 = K.sum((y_true +  y_pred), axis = (0,1,2))     
    '''
    num2 = K.sum(num1[1:])
    dnm2 = K.sum(dnm1[1:])
    '''
    '''
    num2 = K.sum(num1[1]+num1[2]+num1[4])
    dnm2 = K.sum(dnm1[1]+dnm1[2]+dnm1[4])
    '''
    #num2 = K.sum(num1)
    #dnm2 = K.sum(dnm1)


    #num2 = K.sum(((1-y_true) * (1- y_pred)), axis = (0,1,2,3))
    #dnm2 = K.sum((2- y_true -  y_pred), axis = (0,1,2,3))     
    
    f1 = ((2*num1 + smooth1)/ (dnm1 + smooth1)) #+ w1*((2*num2 + smooth1)/ (dnm2 + smooth1)))/(w1+w2)
    #f1 = K.sum((2*num2 + smooth1)/ (dnm2 + smooth1))
    return K.sum(f1[1:])#+f1[2]+f1[4]+f1[8]+f1[9])
 
def dice_loss(y_true, y_pred):
    ncl = 9.0
    return (ncl-dice(y_true, y_pred))

def comb_acc(y_true, y_pred):
    return (dice(y_true, y_pred) + iou(y_true, y_pred))/2

def comb_loss(y_true, y_pred):
    return (dice_loss(y_true, y_pred))


In [ ]:
size = 256
def unet(size, lri, input_height = size, input_width = size, nClasses = 10):

    K.set_image_data_format('channels_last')
    
    input_size = (input_width, input_height, 6)
    input1 = Input(shape = input_size)
    n = 64
    drate1 = 0
    drate2 = 0
    drate3 = 0
    drate4 = 0
    drate5 = 0
    lmbd = 0
    #bn01 = ( BatchNormalization())(input1) 
    
    #drop0 = Dropout(0)(input1)
    conv1 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer = 'he_normal')(input1)
    drop1 = Dropout(drate1)(conv1)
    bn11 = ( BatchNormalization())(drop1) 

    conv1 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer = 'he_normal')(bn11)
    drop12 = Dropout(drate2)(conv1)
    bn12 = ( BatchNormalization())(drop12) 

    pool1 = MaxPooling2D(pool_size=(2, 2))(bn12)
    
    conv2 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same',  data_format= "channels_last", kernel_initializer = 'he_normal')(pool1)
    drop2 = Dropout(drate1)(conv2) 
    bn21 = ( BatchNormalization())(drop2)

    conv2 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same',  data_format= "channels_last", kernel_initializer = 'he_normal')(bn21)
    drop22 = Dropout(drate2)(conv2)  
    bn22 = ( BatchNormalization())(drop22)   
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(bn22)
    
    conv3 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer = 'he_normal')(pool2)
    drop3 = Dropout(drate1)(conv3)
    bn31 = ( BatchNormalization())(drop3)

    conv3 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn31)
    drop32 = Dropout(drate2)(conv3)
    bn32 = ( BatchNormalization())(drop32) 
    
    pool3 = MaxPooling2D(pool_size=(2, 2))(bn32)
    
    conv4 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(pool3)
    drop4 = Dropout(drate1)(conv4)
    bn41 = ( BatchNormalization())(drop4) 

    conv4 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn41)
    drop42 = Dropout(drate5)(conv4)
    bn42 = ( BatchNormalization())(drop42) 
    
    pool4 = MaxPooling2D(pool_size=(2, 2))(bn42)
   
    conv5 = Conv2D(n*16, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(pool4)
    drop5 = Dropout(drate5)(conv5)
    bn51 = ( BatchNormalization())(drop5) 
    conv5 = Conv2D(n*16, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn51)
    drop52 = Dropout(drate4)(conv5)
    bn52 = ( BatchNormalization())(drop52) 
    
    up6 = Conv2D(n*8, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same',data_format= "channels_last",  kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn52))
    merge6 = concatenate([bn42,up6], axis = 3)
    #merge6 = concatenate([conv4,up6], axis = 3)
    
    conv6 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge6)
    drop6 = Dropout(drate3)(conv6)
    bn61 = ( BatchNormalization())(drop6) 

    conv6 = Conv2D(n*8, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn61)
    drop62 = Dropout(drate4)(conv6)
    bn62 = ( BatchNormalization())(drop62) 
    
    up7 = Conv2D(n*4, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn62))
    
    merge7 = concatenate([bn32,up7], axis = 3)
    
    
    conv7 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge7)
    drop7 = Dropout(drate3)(conv7)
    bn71 = ( BatchNormalization())(drop7) 

    conv7 = Conv2D(n*4, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn71)
    drop72 = Dropout(drate4)(conv7)
    bn72 = ( BatchNormalization())(drop72) 
    

    up8 = Conv2D(n*2, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn72))
    
    
    merge8 = concatenate([bn22,up8], axis = 3)
    
    conv8 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge8)
    drop8 = Dropout(drate3)(conv8)
    bn81 = ( BatchNormalization())(drop8) 

    conv8 = Conv2D(n*2, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn81)
    drop82 = Dropout(drate4)(conv8)
    bn82 = ( BatchNormalization())(drop82) 
    
    up9 = Conv2D(n, (2,2), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(UpSampling2D(size = (2,2))(bn82))
    merge9 = concatenate([bn12,up9], axis = 3)
    
    conv9 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(merge9)
    drop9 = Dropout(drate3)(conv9)
    bn91 = ( BatchNormalization())(drop9) 

    conv9 = Conv2D(n, (3,3), kernel_regularizer=l2(lmbd), activation = 'relu', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn91)
    drop92 = Dropout(drate4)(conv9)
    bn92 = ( BatchNormalization())(drop92) 
    
    conv10 = Conv2D(nClasses, (1,1), kernel_regularizer=l2(lmbd), activation = 'softmax', padding = 'same', data_format= "channels_last", kernel_initializer ='he_normal')(bn92)
    model = Model(inputs = input1, outputs = conv10)
    #model.compile(optimizer = Adam(lr  = lri), loss= dice_loss, metrics = [iou, dice])
    model.compile(optimizer = Adam(lr  = lri), loss= dice_loss, metrics = [iou, dice])
   
    return model


In [ ]:
filepath1 = '/content/drive/My Drive/Excel/vietnam train images'
filepath2 = '/content/drive/My Drive/Excel/vietnam train labels'

#filepath1 = '/content/drive/My Drive/Excel/vietnam images'
#filepath2 = '/content/drive/My Drive/Excel/vietnam pred final 4class4 processed'
size = 256
#List of all jps and tif files
ipath = ((get_all_images(filepath1, 'img')))
cpath = ((get_all_images(filepath2, 'img')))
print(len(ipath))
print(len(cpath))

ipath_train = ipath
cpath_train = cpath

# ###ipath_train, ipath_test, cpath_train, cpath_test = train_test_split(ipath,cpath, train_size = 0.9, random_state=64, shuffle = True)
# print(len(ipath_train), len(ipath_test))#, len(ipath_val), len(ipath_dev))



23
23


In [ ]:
idj = []
ind = []
ar1 = []
ar2 = []


for i in range(len(ipath_train)):
    idj,ind,ar1,ar2 = apply_coord(ipath_train[i],idj,ind,ar1,ar2,1,i)
    #numbr = (800*nmbr*num[i]).astype(np.uint32)
    #idj,ind,ar1,ar2 = apply_rdata(ipath[i],idj,ind,ar1,ar2,numbr,i)
    # idj,ind,ar1,ar2 = apply_rdata(ipath_train[i],idj,ind,ar1,ar2,10000,i)


7561 7694
29 30 870
(29,)
(30,)
idj:  870
ar1:  870
ar2:  870
10980 7694
42 30 1260
(42,)
(30,)
idj:  1260
ar1:  1260
ar2:  1260
10980 7694
42 30 1260
(42,)
(30,)
idj:  1260
ar1:  1260
ar2:  1260
7561 10980
29 42 1218
(29,)
(42,)
idj:  1218
ar1:  1218
ar2:  1218
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764
ar2:  1764
10980 10980
42 42 1764
(42,)
(42,)
idj:  1764
ar1:  1764


In [ ]:
def shuffle(idj,ind,ar1,ar2):
    for ixd in range(5):
        idj,ind,ar1,ar2 = shuffle_list(idj,ind,ar1,ar2, seeds = (204+ixd))

    return (idj,ind,ar1,ar2)

#idj0,ind0,ar10,ar20 = shuffel(idj0,ind0,ar10,ar20)

ar1_train, ar1_dev, ar2_train, ar2_dev, ind_train, ind_dev, idj_train, idj_dev = train_test_split(ar1,ar2,ind,idj, train_size = 0.9, random_state=64, shuffle = True)#, stratify = idj)
ar1_test, ar1_val, ar2_test, ar2_val, ind_test, ind_val, idj_test, idj_val = train_test_split(ar1_dev,ar2_dev,ind_dev,idj_dev, train_size = 0.5, random_state=64, shuffle = True)#, stratify = idj_dev)

# idj_train,ind_train,ar1_train,ar2_train = shuffle(idj_train,ind_train,ar1_train,ar2_train)
# idj_val,ind_val,ar1_val,ar2_val = shuffle(idj_val,ind_val,ar1_val,ar2_val)

print('ar1_train: ', len(ar1_train))
print('ar1_val: ', len(ar1_val))

print(len(set(idj_train)))
print(len(set(idj_val)))
print(len(set(idj_test)))



ar1_train:  33328
ar1_val:  1852
23
23
23


In [ ]:
print(len(ar1_dev),len(ar2_dev),len(ind_dev),len(idj_dev))

3704 3704 3704 3704


In [ ]:
batch = 128
val_batch = 32
# train_generator = img_generator(ipath_train,cpath_train,idj_train, ind_train,ar1_train,ar2_train, batch)
train_generator = msk_generator(ipath_train,cpath_train,idj_train, ind_train,ar1_train,ar2_train, batch)

# %timeit time1 = train_generator


val_generator = msk_generator(ipath_train,cpath_train,idj_val, ind_val,ar1_val,ar2_val, val_batch)

#%timeit time1 = val_generator



MIN_LR = 5e-5
MAX_LR = 2e-4
BATCH_SIZE = batch
STEP_SIZE = 8
CLR_METHOD = "triangular"
NUM_EPOCHS = 96


# clr = CyclicLR(
# 	mode= CLR_METHOD,
# 	base_lr= MIN_LR,
# 	max_lr= MAX_LR,
# 	step_size=  STEP_SIZE * ((len(ar1_train)) // BATCH_SIZE))



/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (128, 256, 256, 6) (6 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (128, 256, 256, 10) (10 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1,

In [ ]:
def training(size):
    K.clear_session()
    model = 0
    del model         
    K.set_image_data_format('channels_last')
    seed = 100
    np.random.seed(seed)
    tf.compat.v1.set_random_seed(seed)
    lri = 1e-4

    model = unet(size, lri)
    model.summary()
    # model = load_model('/content/drive/My Drive/Excel/vietnam_unsup2-1.h5', custom_objects={'dice_loss': dice_loss,'iou': iou, 'dice': dice})
    model.load_weights('/content/drive/My Drive/Excel/vietnam_unsup2-3.h5')
    checkpoint = ModelCheckpoint("/content/drive/My Drive/Excel/vietnam_unsup2-4.h5", monitor='val_dice', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)
    #logdir = os.path.join(logs_base_dir, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    #tensorboard = TensorBoard(log_dir= logdir, histogram_freq=1)
    early = EarlyStopping(monitor='val_iou', min_delta=0, patience=1000, verbose=1, mode='auto')
    history = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//batch), epochs = 1000 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//val_batch))    
    #history = model.fit_generator(train_generator, steps_per_epoch = (len(train_X)/ batch), epochs =300 ,verbose = 2, callbacks = [tensorboard, checkpoint], validation_data= val_generator, validation_steps = (len(val_X)/4), shuffle =True)        
    #for layer in model.layers[:6]:
    #    layer.trainable = False
    #model.compile(optimizer = Adam(lr  = lri), loss= dice_loss, metrics = [iou, dice])

    # %timeit time1 = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//32), epochs = 1 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//32))    
    # history = model.fit(x = (arr[:tvsplit, :,:,:3]), y = (arr[:tvsplit, :,:,3:4]), batch_size = 20, epochs = 100 ,verbose = 2, callbacks = [tensorboard, checkpoint, early], validation_data= ((arr[tvsplit:vtsplit, :,:,:3]), (arr[tvsplit:vtsplit, :,:,3:4])), shuffle =True)        
    # %timeit time2 = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//64), epochs = 1 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//32))    
    # %timeit time3 = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//128), epochs = 1 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//32))    
    # %timeit time4 = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//256), epochs = 1 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//32))    
    # %timeit time5 = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//512), epochs = 1 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//32))    
    # %timeit time6 = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//1024), epochs = 1 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//32))    
    # %timeit time7 = model.fit_generator(train_generator, steps_per_epoch = (len(ar1_train)//2048), epochs = 1 ,verbose = 2, callbacks = [checkpoint, early], validation_data= val_generator, validation_steps = (len(ar1_val)//32))    


training(size)



Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 6) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 3520        input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 256, 256, 64) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         dropout[0][0]                    
_______________________________________________________________________________________

KeyboardInterrupt: ignored

In [ ]:
tr_generator = msk_generator(ipath_train,cpath_train,idj_train, ind_train,ar1_train,ar2_train, val_batch)
test_generator = msk_generator(ipath_train,cpath_train,idj_test, ind_test,ar1_test,ar2_test, val_batch)
lri = 1e-4
model = unet(size, lri)
model.load_weights("/content/drive/My Drive/Excel/vietnam_kmeans_gpu_label1.h5")
# model = load_model('/content/drive/My Drive/Excel/vietnam_unsup2-3.h5', custom_objects={'dice_loss': dice_loss,'iou': iou, 'dice': dice})
#print(l0,i0,d0)
print('set', model.metrics_names)  
l,i,d = model.evaluate_generator(tr_generator, verbose=1, steps=(len(ar1_train)//val_batch))
print('train: ',l,i,d)

l,i,d = model.evaluate_generator(val_generator, verbose=1, steps=(len(ar1_val)//val_batch))
print('val: ',l,i,d)

l,i,d = model.evaluate_generator(test_generator, verbose=1, steps=(len(ar1_test)//val_batch))
print('test: ',l,i,d)

   
K.clear_session()
del model 


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (32, 256, 256, 6) (6 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (32, 256, 256, 10) (10 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:136: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3

set ['loss', 'iou', 'dice']
1041/1041 [==============================] - 31s 30ms/step - loss: 1.0148 - iou: 7.9707 - dice: 7.9853
train:  1.014756679534912 7.97073221206665 7.98532247543335
57/57 [==============================] - 2s 29ms/step - loss: 3.3386 - iou: 5.6307 - dice: 5.6614
val:  3.3386118412017822 5.6306562423706055 5.6613874435424805
57/57 [==============================] - 2s 30ms/step - loss: 3.0099 - iou: 5.9901 - dice: 5.9901
test:  3.009887933731079 5.990111827850342 5.990111827850342
